In [43]:
from langchain_community.document_loaders import UnstructuredURLLoader

In [62]:
urls = ['https://www.victoriaonmove.com.au/local-removalists.html','https://victoriaonmove.com.au/index.html']
loader=UnstructuredURLLoader(urls=urls)
data=loader.load()

In [63]:
data

[Document(metadata={'source': 'https://www.victoriaonmove.com.au/local-removalists.html'}, page_content='Loading...\n\nLOCAL REMOVALS\n\nYour trusted partner in seamless moving and packing solutions!\n\nGoogle logo G\n\nGoogle Rating\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\n5 stars, 111 reviews\n\n\n\nRequst A call for You:\n\nLocal removal services via "Victoria on move"\n\nVictoria on Move is your trusted local moving company in Melbourne, specializing in seamless relocation services. As experienced furniture movers and relocation experts, we provide top-notch packing and moving services tailored to your needs. Whether you\'re moving across town or relocating interstate, our professional movers ensure a stress-free experience. Count on Victoria on Move for reliable removal services, making us the preferred choice among local movers in Melbourne. Discover why we\'re recognized for our commitment to quality and customer satisfaction.\

In [64]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000)
docs=text_splitter.split_documents(data)

print("Total documents: ",len(docs))

Total documents:  8


In [65]:
docs[0]

Document(metadata={'source': 'https://www.victoriaonmove.com.au/local-removalists.html'}, page_content='Loading...\n\nLOCAL REMOVALS\n\nYour trusted partner in seamless moving and packing solutions!\n\nGoogle logo G\n\nGoogle Rating\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\n5 stars, 111 reviews\n\n\n\nRequst A call for You:\n\nLocal removal services via "Victoria on move"\n\nVictoria on Move is your trusted local moving company in Melbourne, specializing in seamless relocation services. As experienced furniture movers and relocation experts, we provide top-notch packing and moving services tailored to your needs. Whether you\'re moving across town or relocating interstate, our professional movers ensure a stress-free experience. Count on Victoria on Move for reliable removal services, making us the preferred choice among local movers in Melbourne. Discover why we\'re recognized for our commitment to quality and customer satisfaction.\n

In [66]:
docs[7]

Document(metadata={'source': 'https://victoriaonmove.com.au/index.html'}, page_content="Unlike some other moving companies, we value the trust our customers place in us. Our customer reviews on Google are genuine, and we never manipulate them. We trust in our quality and experience and believe in our destiny to provide top-notch moving and packing services to our clients.\n\nContact Us\n\nOur Clients Say!\n\nDiscover firsthand experiences from our valued clients through their heartfelt testimonials. From seamless moves to exceptional service, our customers share how we've made their relocation journey stress-free and rewarding. Explore their stories and see why they trust us with their moves time and again.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nGet In Touch\n\nWollert Victoria\n\n0404922328\n\nvictoriaonmove07@gmail.com\n\nQuick Links\n\nAbout Us Contact Us Our Services Terms & Condition\n\nPhoto Gallery\n\n\n\n\n\n\n\n\n\n\n\n\n\nCheck us out on Google!\n\nreviews\n\n© Victoria O

In [49]:
from langchain_chroma import Chroma
from langchain_mistralai import MistralAIEmbeddings
from langchain_mistralai import ChatMistralAI

In [50]:
from dotenv import load_dotenv
load_dotenv()

True

In [51]:
import os
os.environ["MISTRAL_API_KEY"] = os.getenv("MISTRAL_API_KEY")

In [73]:
vectorstore=Chroma.from_documents(documents=docs,embedding=MistralAIEmbeddings(),persist_directory='db')

/workspaces/codespaces-blank/venv/lib/python3.10/site-packages/langchain_mistralai/embeddings.py:181: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


In [68]:
retriever=vectorstore.as_retriever(search_type="similarity",
                                   search_kwargs={"k":3})

In [70]:
retrieved_docs=retriever.invoke("what kind of services to they provide?")

In [71]:
retrieved_docs

[Document(id='48cfede9-b9fe-4911-b90c-c0df21b6b857', metadata={'source': 'https://www.victoriaonmove.com.au/local-removalists.html'}, page_content='Icon\n\nApartment Moving\n\nEfficient and careful relocation services tailored for apartments of all sizes, ensuring smooth transitions to your new home.\n\nIcon\n\nVilla Moving\n\nComprehensive moving solutions for large residences and villas, handling valuable possessions with utmost care and precision.\n\nIcon\n\nHousehold Moving\n\nFull-service moving options for households, including packing, loading, transportation, and unpacking services to simplify your move.\n\nIcon\n\nOffice Moving\n\nSpecialized expertise in office relocations, minimizing downtime and ensuring your business operations continue seamlessly.\n\nIcon\n\nFurniture Moving\n\nExperienced in handling furniture of all sizes and types, ensuring safe transport and setup in your new location.\n\nIcon\n\nPacking and Unpacking Services\n\nOptional packing and unpacking service

In [56]:
llm = ChatMistralAI(
model="mistral-small-latest",
temperature=0.4,
max_tokens=500
)

In [57]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt=(
    """
    You are an assistant for  question-answering tasks.
    Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, say that you don't know.
    Use 3 sentences maximum and keep answer concise.
    \n\n
    {context}
    """
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

In [58]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [59]:
response=rag_chain.invoke({"input":"What kind of services do they offer?"})
print(response["answer"])

The services offered include 24/7 reception, free high-speed Wi-Fi, laundry, power backup, food and beverages, and buffet facilities. They also provide moving and packing solutions with various truck sizes tailored to different moving needs.
